# X-Ray Tube Spectrum Simulator
### Interactive tool for X-ray spectrum analysis
---
**Loading simulator... Please wait 5-10 seconds**

In [ ]:
# @title
# ============================================================================
# SIMULADOR DE ESPECTRO DE TUBO DE RAYOS X - VERSIÓN AVANZADA
# Layout de 2 columnas optimizado para visualización
# ============================================================================

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from io import BytesIO
import base64

# ============================================================================
# FUNCIONES FÍSICAS
# ============================================================================

def k_edge(Z):
    """
    Estima la energía del borde K en función del número atómico Z.
    Ajuste polinómico de 4º orden de datos NIST.

    Parameters:
    -----------
    Z : int o float
        Número atómico del elemento

    Returns:
    --------
    float : Energía del borde K en keV
    """
    Z1 = Z
    Z2 = Z1 * Z1
    Z3 = Z2 * Z1
    Z4 = Z3 * Z1

    Ek = (5.822800E-07 * Z4 - 5.966316E-05 * Z3 +
          1.559167E-02 * Z2 - 1.347932E-01 * Z1 + 8.249124E-01)

    return Ek


def k_disc(Z):
    """
    Estima el valor de la discontinuidad en el borde K.

    Parameters:
    -----------
    Z : int o float
        Número atómico del elemento

    Returns:
    --------
    float : Factor de discontinuidad
    """
    Z1 = Z
    Z2 = Z1 * Z1
    Z3 = Z2 * Z1
    Z4 = Z3 * Z1

    disc = (1.649343E-07 * Z4 - 4.918010E-05 * Z3 +
            5.524608E-03 * Z2 - 3.253348E-01 * Z1 + 1.320899E+01)

    return disc


def k_transitions(Z):
    """
    Estima las energías de las transiciones K (Ka1, Ka2, Kb1, Kb2).
    Ajuste polinómico de 4º orden de datos NIST.

    Parameters:
    -----------
    Z : int o float
        Número atómico del elemento

    Returns:
    --------
    tuple : (Eka1, Eka2, Ekb1, Ekb2) en keV
    """
    Z1 = Z
    Z2 = Z1 * Z1
    Z3 = Z2 * Z1
    Z4 = Z3 * Z1

    Eka1 = (5.771237E-07 * Z4 - 6.284852E-05 * Z3 +
            1.351487E-02 * Z2 - 9.674912E-02 * Z1 + 5.966164E-01)

    Eka2 = (3.951093E-07 * Z4 - 4.088840E-05 * Z3 +
            1.231517E-02 * Z2 - 6.967267E-02 * Z1 + 3.980824E-01)

    Ekb1 = (6.037981E-07 * Z4 - 6.893569E-05 * Z3 +
            1.585793E-02 * Z2 - 1.466261E-01 * Z1 + 1.046658E+00)

    Ekb2 = (9.062123E-07 * Z4 - 1.476653E-04 * Z3 +
            2.382930E-02 * Z2 - 4.615298E-01 * Z1 + 5.414789E+00)

    return Eka1, Eka2, Ekb1, Ekb2


def raw_spectrum(E, Z, Emax, Eres=0.1, Im0=1.0, Em0=160, show_components='all'):
    """
    Genera el espectro de rayos X (Bremsstrahlung + característico).

    Parameters:
    -----------
    E : array-like
        Energías a evaluar (keV)
    Z : int
        Número atómico del ánodo
    Emax : float
        Energía máxima (kVp)
    Eres : float
        Resolución energética (keV)
    Im0 : float
        Intensidad máxima de referencia
    Em0 : float
        Energía máxima de referencia
    show_components : str
        'all', 'bremsstrahlung', 'characteristic'

    Returns:
    --------
    tuple : (spectrum_total, spectrum_bremss, spectrum_char)
    """
    E = np.asarray(E)

    # Intensidad máxima normalizada
    Imax = Emax / Em0 * Im0
    Imax = Imax * (Z / 74)

    # Bremsstrahlung (ley de Kramers)
    Ibrem = Imax * (Emax - E) / Emax
    Ibrem = np.where(E <= Emax, Ibrem, 0)

    # Radiación característica
    Ichar = np.zeros_like(E)

    Ek = k_edge(Z)

    if Emax > Ek:
        Eka1, Eka2, Ekb1, Ekb2 = k_transitions(Z)

        # Amplitud del pico Ka1
        Ika1 = Imax / Emax * np.power(Emax - Ek, 1.63)

        # Añadir picos característicos
        mask_ka1 = np.abs(E - Eka1) <= Eres / 2
        mask_ka2 = np.abs(E - Eka2) <= Eres / 2
        mask_kb1 = np.abs(E - Ekb1) <= Eres / 2
        mask_kb2 = np.abs(E - Ekb2) <= Eres / 2

        Ichar[mask_ka1] = np.maximum(Ichar[mask_ka1], Ika1)
        Ichar[mask_ka2] = np.maximum(Ichar[mask_ka2], 0.5714 * Ika1)
        Ichar[mask_kb1] = np.maximum(Ichar[mask_kb1], 0.3411 * Ika1)
        Ichar[mask_kb2] = np.maximum(Ichar[mask_kb2], 0.0897 * Ika1)

    # Seleccionar componentes a mostrar
    if show_components == 'bremsstrahlung':
        return np.maximum(Ibrem, 0), Ibrem, np.zeros_like(E)
    elif show_components == 'characteristic':
        return np.maximum(Ichar, 0), np.zeros_like(E), Ichar
    else:
        return np.maximum(Ibrem + Ichar, 0), Ibrem, Ichar


def filt(E, Z, rho, thickness, E0=100, Z0=20, tau0=0.085, sigma0=0.2):
    """
    Calcula la atenuación producida por un filtro.

    Parameters:
    -----------
    E : array-like
        Energías (keV)
    Z : int
        Número atómico del filtro
    rho : float
        Densidad del filtro (g/cm³)
    thickness : float
        Espesor del filtro (cm)
    E0, Z0, tau0, sigma0 : float
        Parámetros de referencia para el coeficiente de atenuación

    Returns:
    --------
    array : Factor de atenuación (0-1)
    """
    if Z == 0 or thickness == 0:
        return np.ones_like(E)

    E = np.asarray(E)

    # Evitar división por cero: usar un mínimo de energía
    E_safe = np.where(E > 0, E, 1e-6)

    # Coeficiente de atenuación másico
    mu_rho = (tau0 * np.power(Z / Z0, 3.0) * np.power(E0 / E_safe, 3.0) +
              sigma0 * (Z / Z0) * (E0 / E_safe))

    # Aplicar discontinuidad del borde K
    Ek = k_edge(Z)
    below_edge = E < Ek
    mu_rho[below_edge] = mu_rho[below_edge] / k_disc(Z)

    # Atenuación exponencial (donde E=0, atenuación=1)
    attenuation = np.where(E > 0, np.exp(-mu_rho * rho * thickness), 1.0)

    return attenuation


def get_mu_rho(E, Z, E0=100, Z0=20, tau0=0.085, sigma0=0.2):
    """
    Calcula el coeficiente de atenuación másico μ/ρ.

    Parameters:
    -----------
    E : array-like
        Energías (keV)
    Z : int
        Número atómico
    E0, Z0, tau0, sigma0 : float
        Parámetros de referencia

    Returns:
    --------
    array : μ/ρ en cm²/g
    """
    E = np.asarray(E)
    E_safe = np.where(E > 0, E, 1e-6)

    mu_rho = (tau0 * np.power(Z / Z0, 3.0) * np.power(E0 / E_safe, 3.0) +
              sigma0 * (Z / Z0) * (E0 / E_safe))

    # Aplicar discontinuidad del borde K
    Ek = k_edge(Z)
    below_edge = E < Ek
    mu_rho[below_edge] = mu_rho[below_edge] / k_disc(Z)

    return mu_rho


def generate_spectrum(E, Z, kVp, mAs,
                      F0_Z, F0_rho, F0_thick,
                      F1_Z, F1_rho, F1_thick,
                      ripple=0, show_components='all',
                      mAs0=100):
    """
    Genera el espectro completo con filtros y ripple.

    Parameters:
    -----------
    E : array
        Energías a evaluar
    Z : int
        Z del ánodo
    kVp : float
        Voltaje pico
    mAs : float
        Producto corriente × tiempo
    F0_Z, F0_rho, F0_thick : float
        Parámetros filtro intrínseco
    F1_Z, F1_rho, F1_thick : float
        Parámetros filtro adicional
    ripple : float
        Porcentaje de ripple (0-100)
    show_components : str
        Componentes a mostrar
    mAs0 : float
        mAs de referencia

    Returns:
    --------
    tuple : (spectrum_total, spectrum_bremss, spectrum_char)
    """
    if ripple == 0:
        # Sin ripple
        spectrum_total, spectrum_bremss, spectrum_char = raw_spectrum(E, Z, kVp, show_components=show_components)
        spectrum_total = spectrum_total * filt(E, F0_Z, F0_rho, F0_thick)
        spectrum_total = spectrum_total * filt(E, F1_Z, F1_rho, F1_thick)
        spectrum_total = spectrum_total * (mAs / mAs0)

        spectrum_bremss = spectrum_bremss * filt(E, F0_Z, F0_rho, F0_thick)
        spectrum_bremss = spectrum_bremss * filt(E, F1_Z, F1_rho, F1_thick)
        spectrum_bremss = spectrum_bremss * (mAs / mAs0)

        spectrum_char = spectrum_char * filt(E, F0_Z, F0_rho, F0_thick)
        spectrum_char = spectrum_char * filt(E, F1_Z, F1_rho, F1_thick)
        spectrum_char = spectrum_char * (mAs / mAs0)
    else:
        # Con ripple - integrar sobre varios niveles
        nc = 16
        spectrum_total = np.zeros_like(E)
        spectrum_bremss = np.zeros_like(E)
        spectrum_char = np.zeros_like(E)

        for ic in range(nc):
            V_ripp = (1 + ripple / 100 / 2 * (np.cos(ic / nc * np.pi) - 1)) * kVp

            spec_tmp_total, spec_tmp_bremss, spec_tmp_char = raw_spectrum(E, Z, V_ripp, show_components=show_components)

            spec_tmp_total = spec_tmp_total * filt(E, F0_Z, F0_rho, F0_thick)
            spec_tmp_total = spec_tmp_total * filt(E, F1_Z, F1_rho, F1_thick)

            spec_tmp_bremss = spec_tmp_bremss * filt(E, F0_Z, F0_rho, F0_thick)
            spec_tmp_bremss = spec_tmp_bremss * filt(E, F1_Z, F1_rho, F1_thick)

            spec_tmp_char = spec_tmp_char * filt(E, F0_Z, F0_rho, F0_thick)
            spec_tmp_char = spec_tmp_char * filt(E, F1_Z, F1_rho, F1_thick)

            spectrum_total += spec_tmp_total / nc
            spectrum_bremss += spec_tmp_bremss / nc
            spectrum_char += spec_tmp_char / nc

        spectrum_total = spectrum_total * (mAs / mAs0)
        spectrum_bremss = spectrum_bremss * (mAs / mAs0)
        spectrum_char = spectrum_char * (mAs / mAs0)

    return spectrum_total, spectrum_bremss, spectrum_char


def generate_ripple_waveform(kVp, ripple, t_range=(-1, 11), dt=0.05):
    """
    Genera la forma de onda del voltaje con ripple.

    Parameters:
    -----------
    kVp : float
        Voltaje pico
    ripple : float
        Porcentaje de ripple
    t_range : tuple
        Rango de tiempo (ms)
    dt : float
        Paso de tiempo

    Returns:
    --------
    tuple : (t, V_no_ripple, V_ripple)
    """
    t = np.arange(t_range[0], t_range[1], dt)

    V_no_ripple = np.zeros_like(t)
    V_ripple = np.zeros_like(t)

    t0, t1 = 0, 10

    for i, ti in enumerate(t):
        if ti < t0:
            V_no_ripple[i] = 0
            V_ripple[i] = 0
        elif ti <= t1:
            V_no_ripple[i] = kVp
            V_ripple[i] = (1 + ripple / 100 / 2 * (np.cos(ti * np.pi) - 1)) * kVp
        else:
            V_no_ripple[i] = 0
            V_ripple[i] = 0

    return t, V_no_ripple, V_ripple


# ============================================================================
# FUNCIONES DE CÁLCULO DE MÉTRICAS
# ============================================================================

def calculate_mean_energy(E, spectrum):
    """Calcula la energía media del espectro."""
    total_intensity = np.trapezoid(spectrum, E)
    if total_intensity == 0:
        return 0.0
    E_mean = np.trapezoid(E * spectrum, E) / total_intensity
    return E_mean


def calculate_hvl(E, spectrum, filter_Z=13, filter_rho=2.7):
    """Calcula la Capa Hemirreductora (CHR) en mm de Al."""
    initial_kerma = np.trapezoid(spectrum * E, E)

    if initial_kerma == 0:
        return 0.0

    thicknesses_cm = np.linspace(0, 1.0, 200)

    for thick_cm in thicknesses_cm:
        attenuated_spectrum = spectrum * filt(E, filter_Z, filter_rho, thick_cm)
        current_kerma = np.trapezoid(attenuated_spectrum * E, E)

        if current_kerma <= initial_kerma / 2:
            return thick_cm * 10

    return thicknesses_cm[-1] * 10


def calculate_effective_energy(E, spectrum, filter_Z=13, filter_rho=2.7):
    """Calcula la energía efectiva del haz."""
    hvl_mm = calculate_hvl(E, spectrum, filter_Z, filter_rho)
    hvl_cm = hvl_mm / 10

    if hvl_cm == 0:
        return 0.0

    mu_at_hvl = np.log(2) / hvl_cm
    E_test = np.linspace(20, 150, 500)

    for E_eff in E_test:
        mu_rho = (0.085 * np.power(filter_Z / 20, 3.0) * np.power(100 / E_eff, 3.0) +
                  0.2 * (filter_Z / 20) * (100 / E_eff))
        mu_calc = mu_rho * filter_rho

        if mu_calc <= mu_at_hvl:
            return E_eff

    return E_test[-1]


def calculate_total_yield(E, spectrum):
    """Calcula el rendimiento total."""
    return np.trapezoid(spectrum, E)


def calculate_homogeneity_coefficient(E, spectrum, filter_Z=13, filter_rho=2.7):
    """
    Calcula el coeficiente de homogeneidad K₀.₅/K₀.₁.
    """
    initial_kerma = np.trapezoid(spectrum * E, E)

    if initial_kerma == 0:
        return 0.0

    # Buscar CHR (K₀.₅)
    hvl_mm = calculate_hvl(E, spectrum, filter_Z, filter_rho)

    # Buscar espesor que da 10% (K₀.₁)
    thicknesses_cm = np.linspace(0, 2.0, 400)
    tvl_mm = 0

    for thick_cm in thicknesses_cm:
        attenuated_spectrum = spectrum * filt(E, filter_Z, filter_rho, thick_cm)
        current_kerma = np.trapezoid(attenuated_spectrum * E, E)

        if current_kerma <= initial_kerma * 0.1:
            tvl_mm = thick_cm * 10
            break

    if tvl_mm == 0:
        return 0.0

    # Coeficiente de homogeneidad
    h_coeff = hvl_mm / tvl_mm

    return h_coeff


def calculate_energy_distribution(E, spectrum):
    """
    Calcula la distribución de fotones por rango de energía.

    Returns:
    --------
    dict : {'low': %, 'medium': %, 'high': %}
    """
    total = np.trapezoid(spectrum, E)

    if total == 0:
        return {'low': 0, 'medium': 0, 'high': 0}

    # Definir rangos
    mask_low = E < 30
    mask_medium = (E >= 30) & (E < 60)
    mask_high = E >= 60

    low_pct = np.trapezoid(spectrum[mask_low], E[mask_low]) / total * 100
    medium_pct = np.trapezoid(spectrum[mask_medium], E[mask_medium]) / total * 100
    high_pct = np.trapezoid(spectrum[mask_high], E[mask_high]) / total * 100

    return {'low': low_pct, 'medium': medium_pct, 'high': high_pct}


def calculate_characteristic_percentage(E, spectrum_total, spectrum_char):
    """
    Calcula el porcentaje de radiación característica vs Bremsstrahlung.
    """
    total_intensity = np.trapezoid(spectrum_total, E)
    char_intensity = np.trapezoid(spectrum_char, E)

    if total_intensity == 0:
        return 0.0, 0.0

    char_pct = (char_intensity / total_intensity) * 100
    bremss_pct = 100 - char_pct

    return char_pct, bremss_pct


# ============================================================================
# INTERFAZ GRÁFICA CON WIDGETS - LAYOUT 2 COLUMNAS
# ============================================================================

class XRaySpectrumSimulator:
    """Simulador interactivo de espectro de tubo de rayos X."""

    def __init__(self):
        # Presets de Tubo Canónico
        self.presets = {
            'Mamografía (Mo, 28 kV)': {
                'Z': 42, 'kVp': 28, 'mAs': 100,
                'F1_Z': 42, 'F1_rho': 10.2, 'F1_thick': 0.003
            },
            'Radiología General (W, 80 kV)': {
                'Z': 74, 'kVp': 80, 'mAs': 100,
                'F1_Z': 13, 'F1_rho': 2.7, 'F1_thick': 0.0
            },
            'TC (W, 120 kV)': {
                'Z': 74, 'kVp': 120, 'mAs': 100,
                'F1_Z': 13, 'F1_rho': 2.7, 'F1_thick': 0.02
            },
            'Manual': {
                'Z': 74, 'kVp': 80, 'mAs': 100,
                'F1_Z': 13, 'F1_rho': 2.7, 'F1_thick': 0.0
            }
        }

        # Parámetros del Tubo 0 (canónico)
        self.T0_show = True
        self.T0_preset = 'Radiología General (W, 80 kV)'
        self.T0_Z = 74
        self.T0_kVp = 80.0
        self.T0_mAs = 100
        self.T0_F0_Z = 4
        self.T0_F0_rho = 1.8
        self.T0_F0_thick = 0.02
        self.T0_F1_Z = 13
        self.T0_F1_rho = 2.7
        self.T0_F1_thick = 0.0

        # Parámetros del Tubo 1
        self.T1_show = True
        self.T1_Z = 74
        self.T1_kVp = 80.0
        self.T1_mAs = 100
        self.T1_F0_Z = 4
        self.T1_F0_rho = 1.8
        self.T1_F0_thick = 0.02
        self.T1_F1_Z = 13
        self.T1_F1_rho = 2.7
        self.T1_F1_thick = 0.0

        # Ripple
        self.ripple = 0
        self.show_ripple = False

        # Visualización
        self.show_components = 'all'
        self.zoom_E = 100
        self.show_metrics = False
        self.show_advanced = False
        self.show_annotations = False
        self.show_differential = False
        self.show_k_edge = False

        # Marcadores de energía
        self.mark_e_mean = False
        self.mark_e_eff = False
        self.mark_characteristic = False
        self.mark_k_edge = False

        # Materiales predefinidos
        self.anodes = {
            'Wolframio (W)': 74,
            'Molibdeno (Mo)': 42,
            'Rodio (Rh)': 45
        }

        self.filters = {
            'Aluminio': {'Z': 13, 'rho': 2.7},
            'Cobre': {'Z': 29, 'rho': 9.0},
            'Molibdeno': {'Z': 42, 'rho': 10.2},
            'Rodio': {'Z': 45, 'rho': 12.4}
        }

        self.create_widgets()
        self.create_interface()

    def create_widgets(self):
        """Crea todos los widgets de control."""

        # ===== TUBO 0 (CANÓNICO) =====
        self.w_T0_show = widgets.Checkbox(
            value=True, description='Mostrar Tubo Canónico'
        )

        self.w_T0_preset = widgets.Dropdown(
            options=list(self.presets.keys()),
            value='Radiología General (W, 80 kV)',
            description='Preset:',
            style={'description_width': '60px'}
        )

        # Controles manuales (solo visibles si preset = Manual)
        self.w_T0_anode = widgets.Dropdown(
            options=list(self.anodes.keys()),
            value='Wolframio (W)',
            description='Ánodo:',
            layout=widgets.Layout(display='none'),
            style={'description_width': '60px'}
        )

        self.w_T0_kVp = widgets.IntSlider(
            value=80, min=20, max=150, step=1,
            description='kVp:',
            layout=widgets.Layout(display='none'),
            style={'description_width': '60px'}
        )

        self.w_T0_mAs = widgets.IntSlider(
            value=100, min=0, max=200, step=1,
            description='mAs:',
            layout=widgets.Layout(display='none'),
            style={'description_width': '60px'}
        )

        self.w_T0_F1_thick = widgets.FloatSlider(
            value=0.0, min=0, max=0.2, step=0.001,
            description='Filtro (cm):',
            readout_format='.4f',
            layout=widgets.Layout(display='none'),
            style={'description_width': '60px'}
        )

        # ===== TUBO 1 =====
        self.w_T1_show = widgets.Checkbox(
            value=True, description='Mostrar Tubo 1'
        )

        self.w_T1_anode = widgets.Dropdown(
            options=list(self.anodes.keys()),
            value='Wolframio (W)',
            description='Ánodo:',
            style={'description_width': '60px'}
        )

        self.w_T1_kVp = widgets.IntSlider(
            value=80, min=20, max=150, step=1,
            description='kVp:',
            style={'description_width': '60px'}
        )

        self.w_T1_mAs = widgets.IntSlider(
            value=100, min=0, max=200, step=1,
            description='mAs:',
            style={'description_width': '60px'}
        )

        self.w_T1_filter = widgets.Dropdown(
            options=list(self.filters.keys()),
            value='Aluminio',
            description='Material:',
            style={'description_width': '60px'}
        )

        self.w_T1_F1_thick = widgets.FloatSlider(
            value=0.0, min=0, max=0.2, step=0.001,
            description='Espesor (cm):',
            readout_format='.4f',
            style={'description_width': '60px'}
        )

        # ===== RIPPLE =====
        self.w_ripple_show = widgets.Checkbox(
            value=False, description='Mostrar Ripple'
        )

        self.w_ripple = widgets.IntSlider(
            value=0, min=0, max=100, step=1,
            description='Ripple (%):',
            style={'description_width': '60px'}
        )

        # ===== VISUALIZACIÓN =====
        self.w_components = widgets.Dropdown(
            options=[
                ('Bremss + Caract', 'all'),
                ('Solo Bremss', 'bremsstrahlung'),
                ('Solo Caract', 'characteristic')
            ],
            value='all',
            description='Mostrar:',
            style={'description_width': '60px'}
        )

        self.w_zoom = widgets.IntSlider(
            value=100, min=30, max=160, step=1,
            description='Zoom (keV):',
            style={'description_width': '60px'}
        )

        self.w_show_metrics = widgets.Checkbox(
            value=False,
            description='Tabla Métricas'
        )

        self.w_show_advanced = widgets.Checkbox(
            value=False,
            description='Análisis Avanzado'
        )

        self.w_show_annotations = widgets.Checkbox(
            value=False,
            description='Anotar Energías'
        )

        # Checkboxes individuales para marcadores (más compactos)
        self.w_mark_e_mean = widgets.Checkbox(
            value=True,
            description='E media',
            indent=False,
            layout=widgets.Layout(width='auto')
        )

        self.w_mark_e_eff = widgets.Checkbox(
            value=True,
            description='E efectiva',
            indent=False,
            layout=widgets.Layout(width='auto')
        )

        self.w_mark_characteristic = widgets.Checkbox(
            value=True,
            description='Líneas K',
            indent=False,
            layout=widgets.Layout(width='auto')
        )

        self.w_mark_k_edge = widgets.Checkbox(
            value=True,
            description='Borde K',
            indent=False,
            layout=widgets.Layout(width='auto')
        )

        self.w_show_differential = widgets.Checkbox(
            value=False,
            description='Vista Diferencial'
        )

        self.w_show_k_edge = widgets.Checkbox(
            value=False,
            description='Gráfico μ/ρ'
        )

        # Botones de exportación (más compactos)
        self.w_export_csv = widgets.Button(
            description='CSV',
            button_style='info',
            tooltip='Exportar datos a CSV',
            icon='download',
            layout=widgets.Layout(width='80px')
        )

        self.w_export_png = widgets.Button(
            description='PNG',
            button_style='info',
            tooltip='Exportar gráfico',
            icon='image',
            layout=widgets.Layout(width='80px')
        )

        # Asignar callbacks
        self.w_T0_show.observe(self.on_value_change, 'value')
        self.w_T0_preset.observe(self.on_preset_change, 'value')
        self.w_T0_anode.observe(self.on_value_change, 'value')
        self.w_T0_kVp.observe(self.on_value_change, 'value')
        self.w_T0_mAs.observe(self.on_value_change, 'value')
        self.w_T0_F1_thick.observe(self.on_value_change, 'value')

        self.w_T1_show.observe(self.on_value_change, 'value')
        self.w_T1_anode.observe(self.on_value_change, 'value')
        self.w_T1_kVp.observe(self.on_value_change, 'value')
        self.w_T1_mAs.observe(self.on_value_change, 'value')
        self.w_T1_filter.observe(self.on_value_change, 'value')
        self.w_T1_F1_thick.observe(self.on_value_change, 'value')

        self.w_ripple_show.observe(self.on_value_change, 'value')
        self.w_ripple.observe(self.on_value_change, 'value')

        self.w_components.observe(self.on_value_change, 'value')
        self.w_zoom.observe(self.on_value_change, 'value')
        self.w_show_metrics.observe(self.on_value_change, 'value')
        self.w_show_advanced.observe(self.on_value_change, 'value')
        self.w_show_annotations.observe(self.on_value_change, 'value')
        self.w_show_differential.observe(self.on_value_change, 'value')
        self.w_show_k_edge.observe(self.on_value_change, 'value')

        self.w_mark_e_mean.observe(self.on_value_change, 'value')
        self.w_mark_e_eff.observe(self.on_value_change, 'value')
        self.w_mark_characteristic.observe(self.on_value_change, 'value')
        self.w_mark_k_edge.observe(self.on_value_change, 'value')

        self.w_export_csv.on_click(self.export_csv)
        self.w_export_png.on_click(self.export_png)

    def on_preset_change(self, change):
        """Callback cuando cambia el preset del Tubo Canónico."""
        preset_name = change['new']

        if preset_name == 'Manual':
            # Mostrar controles manuales
            self.w_T0_anode.layout.display = 'flex'
            self.w_T0_kVp.layout.display = 'flex'
            self.w_T0_mAs.layout.display = 'flex'
            self.w_T0_F1_thick.layout.display = 'flex'
        else:
            # Ocultar controles manuales
            self.w_T0_anode.layout.display = 'none'
            self.w_T0_kVp.layout.display = 'none'
            self.w_T0_mAs.layout.display = 'none'
            self.w_T0_F1_thick.layout.display = 'none'

            # Aplicar preset
            preset = self.presets[preset_name]
            self.T0_Z = preset['Z']
            self.T0_kVp = preset['kVp']
            self.T0_mAs = preset['mAs']
            self.T0_F1_Z = preset['F1_Z']
            self.T0_F1_rho = preset['F1_rho']
            self.T0_F1_thick = preset['F1_thick']

        self.update_plot()

    def create_interface(self):
        """Crea la interfaz completa con layout de 2 columnas."""

        # Título
        title = widgets.HTML(
            value="<h2 style='text-align:center;'>Simulador de Espectro de Rayos X</h2>"
        )

        # ===== COLUMNA IZQUIERDA (CONTROLES) =====

        # Marcadores (sub-panel compacto)
        markers_box = widgets.VBox([
            widgets.HTML("<small><b>Marcadores:</b></small>"),
            self.w_mark_e_mean,
            self.w_mark_e_eff,
            self.w_mark_characteristic,
            self.w_mark_k_edge
        ], layout=widgets.Layout(padding='5px', border='1px solid #ddd'))

        # Pestañas de configuración
        tab_T0 = widgets.VBox([
            self.w_T0_show,
            self.w_T0_preset,
            self.w_T0_anode,
            self.w_T0_kVp,
            self.w_T0_mAs,
            widgets.HTML("<small><b>Filtro Adicional:</b></small>"),
            self.w_T0_F1_thick
        ])

        tab_T1 = widgets.VBox([
            self.w_T1_show,
            self.w_T1_anode,
            self.w_T1_kVp,
            self.w_T1_mAs,
            widgets.HTML("<small><b>Filtro Adicional:</b></small>"),
            self.w_T1_filter,
            self.w_T1_F1_thick
        ])

        tab_ripple = widgets.VBox([
            self.w_ripple_show,
            self.w_ripple
        ])

        tab_view = widgets.VBox([
            self.w_components,
            self.w_zoom,
            widgets.HTML("<hr style='margin:5px 0;'><small><b>Opciones:</b></small>"),
            self.w_show_metrics,
            self.w_show_advanced,
            self.w_show_annotations,
            markers_box,
            self.w_show_differential,
            self.w_show_k_edge
        ])

        tabs = widgets.Tab(children=[tab_T0, tab_T1, tab_ripple, tab_view])
        tabs.set_title(0, 'T. Canónico')
        tabs.set_title(1, 'Tubo 1')
        tabs.set_title(2, 'Ripple')
        tabs.set_title(3, 'Visual.')

        # Botones de exportación
        export_buttons = widgets.HBox([
            self.w_export_csv,
            self.w_export_png
        ], layout=widgets.Layout(justify_content='space-around', padding='10px'))

        # Columna izquierda completa
        left_column = widgets.VBox([
            tabs,
            export_buttons
        ], layout=widgets.Layout(
            width='30%',
            padding='10px',
            border='1px solid #ddd',
            overflow_y='auto',
            max_height='800px'
        ))

        # ===== COLUMNA DERECHA (GRÁFICOS Y MÉTRICAS) =====

        self.output = widgets.Output()
        self.metrics_output = widgets.Output()

        right_column = widgets.VBox([
            self.output,
            self.metrics_output
        ], layout=widgets.Layout(
            width='68%',
            padding='10px',
            overflow_y='auto',
            max_height='800px'
        ))

        # ===== LAYOUT COMPLETO: TÍTULO + 2 COLUMNAS =====

        main_layout = widgets.HBox([
            left_column,
            right_column
        ], layout=widgets.Layout(
            width='100%',
            justify_content='space-between'
        ))

        self.interface = widgets.VBox([
            title,
            main_layout
        ])

    def on_value_change(self, change):
        """Callback cuando cambia un valor."""
        self.update_plot()

    def update_plot(self):
        """Actualiza el gráfico."""

        # Actualizar parámetros desde widgets
        self.T0_show = self.w_T0_show.value
        self.T0_preset = self.w_T0_preset.value

        if self.T0_preset == 'Manual':
            self.T0_Z = self.anodes[self.w_T0_anode.value]
            self.T0_kVp = self.w_T0_kVp.value
            self.T0_mAs = self.w_T0_mAs.value
            self.T0_F1_thick = self.w_T0_F1_thick.value
        else:
            preset = self.presets[self.T0_preset]
            self.T0_Z = preset['Z']
            self.T0_kVp = preset['kVp']
            self.T0_mAs = preset['mAs']
            self.T0_F1_Z = preset['F1_Z']
            self.T0_F1_rho = preset['F1_rho']
            self.T0_F1_thick = preset['F1_thick']

        self.T1_show = self.w_T1_show.value
        self.T1_Z = self.anodes[self.w_T1_anode.value]
        self.T1_kVp = self.w_T1_kVp.value
        self.T1_mAs = self.w_T1_mAs.value
        filter_mat = self.filters[self.w_T1_filter.value]
        self.T1_F1_Z = filter_mat['Z']
        self.T1_F1_rho = filter_mat['rho']
        self.T1_F1_thick = self.w_T1_F1_thick.value

        self.ripple = self.w_ripple.value
        self.show_ripple = self.w_ripple_show.value

        self.show_components = self.w_components.value
        self.zoom_E = self.w_zoom.value
        self.show_metrics = self.w_show_metrics.value
        self.show_advanced = self.w_show_advanced.value
        self.show_annotations = self.w_show_annotations.value
        self.show_differential = self.w_show_differential.value
        self.show_k_edge = self.w_show_k_edge.value

        self.mark_e_mean = self.w_mark_e_mean.value
        self.mark_e_eff = self.w_mark_e_eff.value
        self.mark_characteristic = self.w_mark_characteristic.value
        self.mark_k_edge = self.w_mark_k_edge.value

        # Generar gráfico
        with self.output:
            clear_output(wait=True)
            self.plot_spectra()

        # Generar tabla de métricas si está activada
        with self.metrics_output:
            clear_output(wait=True)
            if self.show_metrics or self.show_advanced:
                self.display_metrics_table()

    def plot_spectra(self):
        """Genera el gráfico de espectros."""

        # Energías
        E = np.linspace(0, 160, 1600)

        # Generar espectros
        spectra_data = []

        if self.T0_show:
            spec_T0, bremss_T0, char_T0 = generate_spectrum(
                E, self.T0_Z, self.T0_kVp, self.T0_mAs,
                self.T0_F0_Z, self.T0_F0_rho, self.T0_F0_thick,
                self.T0_F1_Z, self.T0_F1_rho, self.T0_F1_thick,
                ripple=0, show_components=self.show_components
            )
            spectra_data.append({
                'spec': spec_T0,
                'bremss': bremss_T0,
                'char': char_T0,
                'label': f'T. Canónico (Z={self.T0_Z}, {self.T0_kVp} kV)',
                'color': 'black',
                'Z': self.T0_Z
            })

        if self.T1_show:
            spec_T1, bremss_T1, char_T1 = generate_spectrum(
                E, self.T1_Z, self.T1_kVp, self.T1_mAs,
                self.T1_F0_Z, self.T1_F0_rho, self.T1_F0_thick,
                self.T1_F1_Z, self.T1_F1_rho, self.T1_F1_thick,
                ripple=self.ripple, show_components=self.show_components
            )
            spectra_data.append({
                'spec': spec_T1,
                'bremss': bremss_T1,
                'char': char_T1,
                'label': f'Tubo 1 (Z={self.T1_Z}, {self.T1_kVp} kV, R={self.ripple}%)',
                'color': 'blue',
                'Z': self.T1_Z
            })

        # Determinar número de subplots
        n_plots = 1
        if self.show_ripple:
            n_plots += 1
        if self.show_differential and self.T0_show and self.T1_show:
            n_plots += 1
        if self.show_k_edge:
            n_plots += 1

        # Crear figura (tamaño ajustado para columna derecha)
        if n_plots == 1:
            fig, axes_temp = plt.subplots(figsize=(10, 5))
            axes = [axes_temp]
        elif n_plots == 2:
            fig, axes = plt.subplots(2, 1, figsize=(10, 8))
        elif n_plots == 3:
            fig, axes = plt.subplots(3, 1, figsize=(10, 11))
        else:
            fig, axes = plt.subplots(4, 1, figsize=(10, 14))

        ax_idx = 0
        ax_spec = axes[ax_idx]

        # Plot espectros principales
        for data in spectra_data:
            ax_spec.plot(E, data['spec'], linewidth=2, label=data['label'],
                        color=data['color'], alpha=0.7)

        ax_spec.set_xlabel('Energía (keV)', fontsize=11, fontweight='bold')
        ax_spec.set_ylabel('Intensidad (u.a.)', fontsize=11, fontweight='bold')
        ax_spec.set_title('Espectro de Rayos X', fontsize=12, fontweight='bold')
        ax_spec.set_xlim(0, self.zoom_E)
        ax_spec.set_ylim(bottom=0)
        ax_spec.grid(True, alpha=0.3)
        ax_spec.legend(fontsize=9)

        # Anotaciones de energías clave
        if self.show_annotations and len(spectra_data) > 0:
            for i, data in enumerate(spectra_data):
                E_mean = calculate_mean_energy(E, data['spec'])
                E_eff = calculate_effective_energy(E, data['spec'])

                y_max = np.max(data['spec'])
                color = data['color']
                alpha_val = 0.6

                if self.mark_e_mean and E_mean > 0:
                    ax_spec.axvline(E_mean, color=color, linestyle='--',
                                   linewidth=1.5, alpha=alpha_val)
                    ax_spec.text(E_mean, y_max * 0.9, f'⟨E⟩={E_mean:.1f}',
                                rotation=90, color=color, fontsize=8,
                                verticalalignment='bottom')

                if self.mark_e_eff and E_eff > 0:
                    ax_spec.axvline(E_eff, color=color, linestyle=':',
                                   linewidth=1.5, alpha=alpha_val)
                    ax_spec.text(E_eff, y_max * 0.8, f'Eeff={E_eff:.1f}',
                                rotation=90, color=color, fontsize=8,
                                verticalalignment='bottom')

                if self.mark_characteristic:
                    Ek = k_edge(data['Z'])
                    if self.T0_kVp > Ek or self.T1_kVp > Ek:
                        Ka1, Ka2, Kb1, Kb2 = k_transitions(data['Z'])
                        for energy, name in [(Ka1, 'Kα1'), (Ka2, 'Kα2'),
                                             (Kb1, 'Kβ1'), (Kb2, 'Kβ2')]:
                            if energy < self.zoom_E:
                                ax_spec.axvline(energy, color='gray', linestyle='-.',
                                               linewidth=0.8, alpha=0.4)
                                ax_spec.text(energy, y_max * 0.95, name,
                                            rotation=90, color='gray', fontsize=7,
                                            verticalalignment='bottom')

                if self.mark_k_edge:
                    Ek = k_edge(data['Z'])
                    if Ek < self.zoom_E:
                        ax_spec.axvline(Ek, color='orange', linestyle='-',
                                       linewidth=1.5, alpha=0.5)
                        ax_spec.text(Ek, y_max * 0.7, f'K={Ek:.1f}',
                                    rotation=90, color='orange', fontsize=8,
                                    verticalalignment='bottom')

        ax_idx += 1

        # Plot ripple
        if self.show_ripple:
            ax_rip = axes[ax_idx]
            t, V_no_rip, V_rip = generate_ripple_waveform(self.T1_kVp, self.ripple)
            ax_rip.plot(t, V_no_rip, 'gray', linewidth=2, label='Sin ripple')
            ax_rip.plot(t, V_rip, 'red', linewidth=3, label=f'Con ripple ({self.ripple}%)')
            ax_rip.set_xlabel('Tiempo (ms)', fontsize=11, fontweight='bold')
            ax_rip.set_ylabel('Voltaje (kV)', fontsize=11, fontweight='bold')
            ax_rip.set_title('Forma de onda', fontsize=12, fontweight='bold')
            ax_rip.grid(True, alpha=0.3)
            ax_rip.legend(fontsize=9)
            ax_rip.set_ylim(bottom=0)
            ax_idx += 1

        # Plot diferencial
        if self.show_differential and self.T0_show and self.T1_show:
            ax_diff = axes[ax_idx]
            diff = spec_T1 - spec_T0

            mask_pos = diff >= 0
            mask_neg = diff < 0

            ax_diff.fill_between(E[mask_pos], 0, diff[mask_pos],
                                color='blue', alpha=0.3, label='T1 > T0')
            ax_diff.fill_between(E[mask_neg], 0, diff[mask_neg],
                                color='red', alpha=0.3, label='T1 < T0')
            ax_diff.plot(E, diff, color='black', linewidth=1.5)
            ax_diff.axhline(0, color='black', linestyle='-', linewidth=0.8)

            ax_diff.set_xlabel('Energía (keV)', fontsize=11, fontweight='bold')
            ax_diff.set_ylabel('Diferencia (u.a.)', fontsize=11, fontweight='bold')
            ax_diff.set_title('Vista Diferencial: T1 - T0', fontsize=12, fontweight='bold')
            ax_diff.set_xlim(0, self.zoom_E)
            ax_diff.grid(True, alpha=0.3)
            ax_diff.legend(fontsize=9)
            ax_idx += 1

        # Plot borde K (μ/ρ)
        if self.show_k_edge:
            ax_k = axes[ax_idx]

            E_range = np.linspace(5, 100, 500)
            mu_rho = get_mu_rho(E_range, self.T1_F1_Z)

            ax_k.semilogy(E_range, mu_rho, linewidth=2, color='purple')

            Ek = k_edge(self.T1_F1_Z)
            ax_k.axvline(Ek, color='red', linestyle='--', linewidth=2,
                        label=f'Borde K = {Ek:.1f} keV')

            ax_k.set_xlabel('Energía (keV)', fontsize=11, fontweight='bold')
            ax_k.set_ylabel('μ/ρ (cm²/g)', fontsize=11, fontweight='bold')
            ax_k.set_title(f'μ/ρ - Filtro (Z={self.T1_F1_Z})', fontsize=12, fontweight='bold')
            ax_k.grid(True, alpha=0.3, which='both')
            ax_k.legend(fontsize=9)
            ax_k.set_xlim(5, 100)

        plt.tight_layout()
        plt.show()

    def display_metrics_table(self):
        """Muestra la tabla de métricas."""

        E = np.linspace(0, 160, 1600)

        # Calcular métricas para ambos tubos
        if self.T0_show:
            spec_T0, bremss_T0, char_T0 = generate_spectrum(
                E, self.T0_Z, self.T0_kVp, self.T0_mAs,
                self.T0_F0_Z, self.T0_F0_rho, self.T0_F0_thick,
                self.T0_F1_Z, self.T0_F1_rho, self.T0_F1_thick,
                ripple=0, show_components=self.show_components
            )

            E_mean_T0 = calculate_mean_energy(E, spec_T0)
            hvl_T0 = calculate_hvl(E, spec_T0)
            E_eff_T0 = calculate_effective_energy(E, spec_T0)
            yield_T0 = calculate_total_yield(E, spec_T0)

            if self.show_advanced:
                h_coeff_T0 = calculate_homogeneity_coefficient(E, spec_T0)
                dist_T0 = calculate_energy_distribution(E, spec_T0)
                char_pct_T0, bremss_pct_T0 = calculate_characteristic_percentage(
                    E, spec_T0, char_T0)
        else:
            E_mean_T0 = hvl_T0 = E_eff_T0 = yield_T0 = None
            h_coeff_T0 = dist_T0 = char_pct_T0 = bremss_pct_T0 = None

        if self.T1_show:
            spec_T1, bremss_T1, char_T1 = generate_spectrum(
                E, self.T1_Z, self.T1_kVp, self.T1_mAs,
                self.T1_F0_Z, self.T1_F0_rho, self.T1_F0_thick,
                self.T1_F1_Z, self.T1_F1_rho, self.T1_F1_thick,
                ripple=self.ripple, show_components=self.show_components
            )

            E_mean_T1 = calculate_mean_energy(E, spec_T1)
            hvl_T1 = calculate_hvl(E, spec_T1)
            E_eff_T1 = calculate_effective_energy(E, spec_T1)
            yield_T1 = calculate_total_yield(E, spec_T1)

            if self.show_advanced:
                h_coeff_T1 = calculate_homogeneity_coefficient(E, spec_T1)
                dist_T1 = calculate_energy_distribution(E, spec_T1)
                char_pct_T1, bremss_pct_T1 = calculate_characteristic_percentage(
                    E, spec_T1, char_T1)
        else:
            E_mean_T1 = hvl_T1 = E_eff_T1 = yield_T1 = None
            h_coeff_T1 = dist_T1 = char_pct_T1 = bremss_pct_T1 = None

        # Construir tabla
        rows = []

        # Métricas básicas
        if self.show_metrics:
            row = ['<b>Energía media</b>']
            if E_mean_T0 is not None:
                row.append(f'{E_mean_T0:.2f} keV')
            else:
                row.append('-')

            if E_mean_T1 is not None:
                row.append(f'{E_mean_T1:.2f} keV')
                if E_mean_T0 is not None and E_mean_T0 > 0:
                    delta = (E_mean_T1 / E_mean_T0 - 1) * 100
                    color = "green" if delta > 0 else "red"
                    row.append(f'<span style="color: {color}">{delta:+.1f}%</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>CHR (mm Al)</b>']
            if hvl_T0 is not None:
                row.append(f'{hvl_T0:.2f} mm')
            else:
                row.append('-')

            if hvl_T1 is not None:
                row.append(f'{hvl_T1:.2f} mm')
                if hvl_T0 is not None and hvl_T0 > 0:
                    delta = (hvl_T1 / hvl_T0 - 1) * 100
                    color = "green" if delta > 0 else "red"
                    row.append(f'<span style="color: {color}">{delta:+.1f}%</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>Energía efectiva</b>']
            if E_eff_T0 is not None:
                row.append(f'{E_eff_T0:.2f} keV')
            else:
                row.append('-')

            if E_eff_T1 is not None:
                row.append(f'{E_eff_T1:.2f} keV')
                if E_eff_T0 is not None and E_eff_T0 > 0:
                    delta = (E_eff_T1 / E_eff_T0 - 1) * 100
                    color = "green" if delta > 0 else "red"
                    row.append(f'<span style="color: {color}">{delta:+.1f}%</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>Rendimiento total</b>']
            if yield_T0 is not None:
                row.append(f'{yield_T0:.1f} u.a.')
            else:
                row.append('-')

            if yield_T1 is not None:
                row.append(f'{yield_T1:.1f} u.a.')
                if yield_T0 is not None and yield_T0 > 0:
                    delta = (yield_T1 / yield_T0 - 1) * 100
                    color = "green" if delta > 0 else "red"
                    row.append(f'<span style="color: {color}">{delta:+.1f}%</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

        # Métricas avanzadas
        if self.show_advanced:
            row = ['<b>Coef. homogeneidad</b>']
            if h_coeff_T0 is not None and h_coeff_T0 > 0:
                row.append(f'{h_coeff_T0:.3f}')
            else:
                row.append('-')

            if h_coeff_T1 is not None and h_coeff_T1 > 0:
                row.append(f'{h_coeff_T1:.3f}')
                if h_coeff_T0 is not None and h_coeff_T0 > 0:
                    delta = (h_coeff_T1 / h_coeff_T0 - 1) * 100
                    color = "green" if delta > 0 else "red"
                    row.append(f'<span style="color: {color}">{delta:+.1f}%</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>Fotones <30 keV</b>']
            if dist_T0 is not None:
                row.append(f'{dist_T0["low"]:.1f}%')
            else:
                row.append('-')

            if dist_T1 is not None:
                row.append(f'{dist_T1["low"]:.1f}%')
                if dist_T0 is not None:
                    delta = dist_T1["low"] - dist_T0["low"]
                    color = "red" if delta > 0 else "green"
                    row.append(f'<span style="color: {color}">{delta:+.1f} pp</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>Fotones 30-60 keV</b>']
            if dist_T0 is not None:
                row.append(f'{dist_T0["medium"]:.1f}%')
            else:
                row.append('-')

            if dist_T1 is not None:
                row.append(f'{dist_T1["medium"]:.1f}%')
                if dist_T0 is not None:
                    delta = dist_T1["medium"] - dist_T0["medium"]
                    color = "green" if abs(delta) < 5 else "gray"
                    row.append(f'<span style="color: {color}">{delta:+.1f} pp</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>Fotones >60 keV</b>']
            if dist_T0 is not None:
                row.append(f'{dist_T0["high"]:.1f}%')
            else:
                row.append('-')

            if dist_T1 is not None:
                row.append(f'{dist_T1["high"]:.1f}%')
                if dist_T0 is not None:
                    delta = dist_T1["high"] - dist_T0["high"]
                    color = "green" if delta > 0 else "red"
                    row.append(f'<span style="color: {color}">{delta:+.1f} pp</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

            row = ['<b>Rad. característica</b>']
            if char_pct_T0 is not None:
                row.append(f'{char_pct_T0:.1f}%')
            else:
                row.append('-')

            if char_pct_T1 is not None:
                row.append(f'{char_pct_T1:.1f}%')
                if char_pct_T0 is not None:
                    delta = char_pct_T1 - char_pct_T0
                    color = "blue"
                    row.append(f'<span style="color: {color}">{delta:+.1f} pp</span>')
                else:
                    row.append('-')
            else:
                row.extend(['-', '-'])
            rows.append(row)

        # Construir HTML
        html = '''
        <div style="margin-top: 10px; padding: 10px; background-color: #f8f9fa; border-radius: 8px;">
            <h4 style="margin-top: 0; color: #2c3e50;">📊 Métricas del Espectro</h4>
            <table style="width: 100%; border-collapse: collapse; background-color: white; box-shadow: 0 2px 4px rgba(0,0,0,0.1); font-size: 13px;">
                <thead>
                    <tr style="background-color: #3498db; color: white;">
                        <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Parámetro</th>
                        <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">T0</th>
                        <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">T1</th>
                        <th style="padding: 8px; text-align: center; border: 1px solid #ddd;">Δ</th>
                    </tr>
                </thead>
                <tbody>
        '''

        for i, row in enumerate(rows):
            bg_color = '#f2f2f2' if i % 2 == 0 else 'white'
            html += f'<tr style="background-color: {bg_color};">'
            for j, cell in enumerate(row):
                align = 'left' if j == 0 else 'center'
                html += f'<td style="padding: 6px; text-align: {align}; border: 1px solid #ddd;">{cell}</td>'
            html += '</tr>'

        html += '''
                </tbody>
            </table>
            <p style="margin-top: 8px; font-size: 11px; color: #7f8c8d;">
                <b>Nota:</b> CHR = Capa Hemirreductora. pp = puntos porcentuales.
            </p>
        </div>
        '''

        display(HTML(html))

    def export_csv(self, button):
        """Exporta datos a CSV."""
        E = np.linspace(0, 160, 1600)

        data_dict = {'Energia_keV': E}

        if self.T0_show:
            spec_T0, _, _ = generate_spectrum(
                E, self.T0_Z, self.T0_kVp, self.T0_mAs,
                self.T0_F0_Z, self.T0_F0_rho, self.T0_F0_thick,
                self.T0_F1_Z, self.T0_F1_rho, self.T0_F1_thick,
                ripple=0, show_components=self.show_components
            )
            data_dict['Tubo_Canonico'] = spec_T0

        if self.T1_show:
            spec_T1, _, _ = generate_spectrum(
                E, self.T1_Z, self.T1_kVp, self.T1_mAs,
                self.T1_F0_Z, self.T1_F0_rho, self.T1_F0_thick,
                self.T1_F1_Z, self.T1_F1_rho, self.T1_F1_thick,
                ripple=self.ripple, show_components=self.show_components
            )
            data_dict['Tubo_1'] = spec_T1

            if self.T0_show:
                data_dict['Diferencia_T1_T0'] = spec_T1 - spec_T0

        df = pd.DataFrame(data_dict)
        df.to_csv('espectro_rayos_x.csv', index=False)

        print("✅ Datos exportados a 'espectro_rayos_x.csv'")

    def export_png(self, button):
        """Exporta gráfico como PNG."""
        plt.savefig('espectro_rayos_x.png', dpi=300, bbox_inches='tight')
        print("✅ Gráfico exportado a 'espectro_rayos_x.png'")

    def run(self):
        """Ejecuta el simulador."""
        display(self.interface)
        self.update_plot()


# ============================================================================
# EJECUCIÓN
# ============================================================================

print("="*70)
print("SIMULADOR DE ESPECTRO DE RAYOS X")
print("="*70)
#print("\nLayout optimizado:")
#print("✅ Columna izquierda: Controles (30%)")
#print("✅ Columna derecha: Gráficos y métricas (70%)")
#print("✅ Sin scroll en portátil")
#print("✅ Ideal para proyección en clase")
print("="*70)
#print()

# Crear y ejecutar simulador
simulator = XRaySpectrumSimulator()
simulator.run()

SIMULADOR DE ESPECTRO DE RAYOS X
